<a href="https://colab.research.google.com/github/strixHub/BigDataProject/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://analisi.transparenciacatalunya.cat/Habitatge/Preu-mitj-del-lloguer-d-habitatges-per-municipi/qww9-bvhh/explore/query/SELECT%0A%20%20%60ambit_territorial%60%2C%0A%20%20%60codi_territorial%60%2C%0A%20%20%60nom_territori%60%2C%0A%20%20%60any%60%2C%0A%20%20%60periode%60%2C%0A%20%20%60habitatges%60%2C%0A%20%20%60renda%60%2C%0A%20%20%60tram_preus%60%0AORDER%20BY%20%60any%60%20DESC%20NULL%20LAST/page/filter


https://analisi.transparenciacatalunya.cat/Sector-P-blic/Dades-hist-riques-de-poblaci-dels-municipis-de-Cat/x5sz-niat/explore/query/SELECT%20%60codi_10%60%2C%20%60nom_ens%60%2C%20%60any%60%2C%20%60total%60%2C%20%60homes%60%2C%20%60dones%60/page/filter

https://www.ine.es/jaxiT3/Datos.htm?t=6150#!tabs-tabla


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/csv/Preu_mitj__del_lloguer_d_habitatges_per_municipi.csv')

In [ ]:
df.duplicated().any()

False

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30041 entries, 0 to 30040
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Àmbit territorial  30041 non-null  object 
 1   Codi territorial   30041 non-null  int64  
 2   Nom territori      30041 non-null  object 
 3   Any                30041 non-null  int64  
 4   Període            30041 non-null  object 
 5   Habitatges         30041 non-null  int64  
 6   Renda              17298 non-null  float64
 7   Tram de preus      30041 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 1.8+ MB


In [ ]:
#Elimino la columna de tramo de precios antes de seleccionar los diferentes valores ya que no la necesito
df = df.drop('Tram de preus', axis=1)
numerical_feature = df.select_dtypes(include=["number"]).columns
categorical_feature = df.select_dtypes(include = ["object"]).columns

print(numerical_feature)
print(categorical_feature)

Index(['Codi territorial', 'Any', 'Habitatges', 'Renda'], dtype='object')
Index(['Àmbit territorial', 'Nom territori', 'Període'], dtype='object')


In [ ]:
df[categorical_feature].describe()

,Àmbit territorial,Nom territori,Període
count,30041,30041,30041
unique,1,950,10
top,Municipi,Abrera,gener-desembre
freq,30041,40,11904


In [ ]:
df[numerical_feature].describe()

,Codi territorial,Any,Habitatges,Renda
count,30041.000000,30041.000000,30041.000000,17298.000000
mean,20237.207383,2018.081089,117.446057,545.548369
std,12632.148693,4.179871,1185.854777,183.156686
min,8001.000000,2007.000000,1.000000,90.910000
25%,8211.000000,2016.000000,2.000000,413.912500
50%,17124.000000,2020.000000,8.000000,513.780000
75%,25185.000000,2021.000000,39.000000,639.190000
max,43907.000000,2022.000000,57158.000000,1644.770000


El codigo territorial lo paso a tipo categorical ya que es identificativo

In [ ]:
df['Codi territorial'] = df['Codi territorial'].astype(object)

# **Missing Data**

In [ ]:
print(df.count().to_string())

Àmbit territorial    30041
Codi territorial     30041
Nom territori        30041
Any                  30041
Període              30041
Habitatges           30041
Renda                17298


In [ ]:
df[numerical_feature].info(True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30041 entries, 0 to 30040
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Codi territorial  30041 non-null  object 
 1   Any               30041 non-null  int64  
 2   Habitatges        30041 non-null  int64  
 3   Renda             17298 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 938.9+ KB


In [ ]:
for col in numerical_feature:
  if df[col].isnull().sum()>0:
    print("{}: {} null values and {}% is null values.".format( col, df[col].isnull().sum(), (df[col].isnull().sum()/df[col].size*100) ) )

Renda: 12743 null values and 42.41869445091708% is null values.


In [ ]:
df[categorical_feature].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30041 entries, 0 to 30040
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Àmbit territorial  30041 non-null  object
 1   Nom territori      30041 non-null  object
 2   Període            30041 non-null  object
dtypes: object(3)
memory usage: 704.2+ KB


In [ ]:
for col in categorical_feature:
  if df[col].isnull().sum()>0:
    print("{}: {} null values and {}% is null values.".format( col, df[col].isnull().sum(), (df[col].isnull().sum()/df[col].size*100) ) )

# **Dealing with Missing Data**

**#1 Drop Columns with Most Missing Values**

Elimino todas las filas que no contengan información de la renda media, ya que son demasiadas y poner la media podría influenciar en los resultados.

In [ ]:
df_clean = df.dropna(subset=['Renda'])

In [ ]:
df_clean.describe()

,Any,Habitatges,Renda
count,17298.000000,17298.000000,17298.000000
mean,2017.983929,202.245809,545.548369
std,4.161866,1557.339634,183.156686
min,2007.000000,6.000000,90.910000
25%,2016.000000,12.000000,413.912500
50%,2019.000000,29.000000,513.780000
75%,2021.000000,98.000000,639.190000
max,2022.000000,57158.000000,1644.770000


In [ ]:
df_clean[df_clean.duplicated()]

,Àmbit territorial,Codi territorial,Nom territori,Any,Període,Habitatges,Renda


In [ ]:
dropped_duplicate = df_clean.drop_duplicates()

In [ ]:

dropped_duplicate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17298 entries, 0 to 30040
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Àmbit territorial  17298 non-null  object 
 1   Codi territorial   17298 non-null  object 
 2   Nom territori      17298 non-null  object 
 3   Any                17298 non-null  int64  
 4   Període            17298 non-null  object 
 5   Habitatges         17298 non-null  int64  
 6   Renda              17298 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [ ]:
dropped_duplicate.to_csv("/content/drive/MyDrive/csv/Preu_mitj__del_lloguer_d_habitatges_per_municipi_CLEANED.csv")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/csv/Dades_hist_riques_de_poblaci__dels_municipis_de_Catalunya.csv')

In [ ]:
df.duplicated().any()

False

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32840 entries, 0 to 32839
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CODI_10  32840 non-null  int64  
 1   NOM_ENS  32840 non-null  object 
 2   ANY      32840 non-null  int64  
 3   TOTAL    32840 non-null  int64  
 4   HOMES    12247 non-null  float64
 5   DONES    12247 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.5+ MB


In [ ]:
numerical_feature = df.select_dtypes(include=["number"]).columns
categorical_feature = df.select_dtypes(include = ["object"]).columns

print(numerical_feature)
print(categorical_feature)

Index(['CODI_10', 'ANY', 'TOTAL', 'HOMES', 'DONES'], dtype='object')
Index(['NOM_ENS'], dtype='object')


In [ ]:
df[categorical_feature].describe()

,NOM_ENS
count,32840
unique,1061
top,Ajuntament de Palol de Revardit
freq,32


In [ ]:
df[numerical_feature].describe()

,CODI_10,ANY,TOTAL,HOMES,DONES
count,3.284000e+04,32840.000000,3.284000e+04,1.224700e+04,1.224700e+04
mean,2.548868e+09,2006.370524,2.039214e+04,7.868952e+03,8.076859e+03
std,1.909931e+09,9.579930,1.914623e+05,8.139580e+04,8.539922e+04
min,8.001800e+08,1990.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.259200e+08,1998.000000,2.957500e+02,8.400000e+01,7.300000e+01
50%,1.721680e+09,2007.000000,9.000000e+02,3.150000e+02,2.930000e+02
75%,4.300810e+09,2015.000000,3.935000e+03,1.577000e+03,1.527000e+03
max,8.104240e+09,2022.000000,5.743402e+06,2.791250e+06,2.923480e+06


El CODI_10 lo paso a tipo categorical ya que es identificativo

In [ ]:
df['CODI_10'] = df['CODI_10'].astype(object)

# **Missing Data**

In [ ]:
print(df.count().to_string())

CODI_10    32840
NOM_ENS    32840
ANY        32840
TOTAL      32840
HOMES      12247
DONES      12247


In [ ]:
df[numerical_feature].info(True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32840 entries, 0 to 32839
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CODI_10  32840 non-null  object 
 1   ANY      32840 non-null  int64  
 2   TOTAL    32840 non-null  int64  
 3   HOMES    12247 non-null  float64
 4   DONES    12247 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
for col in numerical_feature:
  if df[col].isnull().sum()>0:
    print("{}: {} null values and {}% is null values.".format( col, df[col].isnull().sum(), (df[col].isnull().sum()/df[col].size*100) ) )

HOMES: 20593 null values and 62.707064555420224% is null values.
DONES: 20593 null values and 62.707064555420224% is null values.


In [ ]:
df[categorical_feature].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32840 entries, 0 to 32839
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   NOM_ENS  32840 non-null  object
dtypes: object(1)
memory usage: 256.7+ KB


In [ ]:
for col in categorical_feature:
  if df[col].isnull().sum()>0:
    print("{}: {} null values and {}% is null values.".format( col, df[col].isnull().sum(), (df[col].isnull().sum()/df[col].size*100) ) )

# **Dealing with Missing Data**

I think both columns homes and dones will not take importance in this

**#1 Drop Columns with Most Missing Values**

In [ ]:
most_missing = [ col for col in df[df.columns] if (df[col].isna().mean()>0.3) ]

df = df.drop(most_missing, axis=1)

In [ ]:
len(most_missing)

2

In [ ]:
df.describe()

,ANY,TOTAL
count,32840.000000,3.284000e+04
mean,2006.370524,2.039214e+04
std,9.579930,1.914623e+05
min,1990.000000,0.000000e+00
25%,1998.000000,2.957500e+02
50%,2007.000000,9.000000e+02
75%,2015.000000,3.935000e+03
max,2022.000000,5.743402e+06


In [ ]:
df[df.duplicated()]

,CODI_10,NOM_ENS,ANY,TOTAL


In [ ]:
df = df.drop_duplicates()

In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32840 entries, 0 to 32839
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CODI_10  32840 non-null  object
 1   NOM_ENS  32840 non-null  object
 2   ANY      32840 non-null  int64 
 3   TOTAL    32840 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.3+ MB


In [ ]:
df.to_csv("/content/drive/MyDrive/csv/Dades_hist_riques_de_poblaci__dels_municipis_de_Catalunya_CLEANED.csv")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/csv/Compraventa.csv')
col_dtypes = {
    'Total': 'str'
}

# Load the CSV with defined column types
df = pd.read_csv('/content/drive/MyDrive/csv/Compraventa.csv', dtype=col_dtypes)
df['Total'] = df['Total'].str.replace('.', '').astype(int)


<ipython-input-394-de51ccd0942f>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Total'] = df['Total'].str.replace('.', '').astype(int)


In [ ]:
df.duplicated().any()

False

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3920 entries, 0 to 3919
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Total Nacional                   3920 non-null   object
 1   Comunidades y Ciudades Autnomas  3920 non-null   object
 2   Provincias                       3920 non-null   object
 3   Rgimen y estado                  3920 non-null   object
 4   Periodo                          3920 non-null   object
 5   Total                            3920 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 183.9+ KB


In [ ]:
df = df.drop('Total Nacional', axis=1)
df = df.drop('Rgimen y estado', axis=1)

In [ ]:
numerical_feature = df.select_dtypes(include=["number"]).columns
categorical_feature = df.select_dtypes(include = ["object"]).columns

print(numerical_feature)
print(categorical_feature)

Index(['Total'], dtype='object')
Index(['Comunidades y Ciudades Autnomas', 'Provincias', 'Periodo'], dtype='object')


In [ ]:
df[categorical_feature].describe()

,Comunidades y Ciudades Autnomas,Provincias,Periodo
count,3920,3920,3920
unique,1,4,196
top,09 Catalua,08 Barcelona,2023M04
freq,3920,980,20


In [ ]:
df[numerical_feature].describe()

,Total
count,3920.000000
mean,850.496939
std,1188.167396
min,0.000000
25%,149.000000
50%,425.000000
75%,875.000000
max,8728.000000


# **Missing Data**

In [ ]:
print(df.count().to_string())

Comunidades y Ciudades Autnomas    3920
Provincias                         3920
Periodo                            3920
Total                              3920


In [ ]:
df[numerical_feature].info(True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3920 entries, 0 to 3919
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Total   3920 non-null   int64
dtypes: int64(1)
memory usage: 30.8 KB


In [ ]:
for col in numerical_feature:
  if df[col].isnull().sum()>0:
    print("{}: {} null values and {}% is null values.".format( col, df[col].isnull().sum(), (df[col].isnull().sum()/df[col].size*100) ) )

In [ ]:
df[categorical_feature].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3920 entries, 0 to 3919
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Comunidades y Ciudades Autnomas  3920 non-null   object
 1   Provincias                       3920 non-null   object
 2   Periodo                          3920 non-null   object
dtypes: object(3)
memory usage: 92.0+ KB


In [ ]:
for col in categorical_feature:
  if df[col].isnull().sum()>0:
    print("{}: {} null values and {}% is null values.".format( col, df[col].isnull().sum(), (df[col].isnull().sum()/df[col].size*100) ) )

# **Dealing with Missing Data**

**#1 Drop Columns with Most Missing Values**

In [ ]:
most_missing = [ col for col in df[df.columns] if (df[col].isna().mean()>0.3) ]

df_clean = df.drop(most_missing, axis=1)

In [ ]:
len(most_missing)

0

In [ ]:
df_clean.describe()

,Total
count,3920.000000
mean,850.496939
std,1188.167396
min,0.000000
25%,149.000000
50%,425.000000
75%,875.000000
max,8728.000000


In [ ]:
df[df.duplicated()]

,Comunidades y Ciudades Autnomas,Provincias,Periodo,Total
2471,09 Catalua,25 Lleida,2013M05,120
2591,09 Catalua,25 Lleida,2019M09,212
2627,09 Catalua,25 Lleida,2016M09,209
2787,09 Catalua,25 Lleida,2019M09,32
2823,09 Catalua,25 Lleida,2016M09,29
3702,09 Catalua,43 Tarragona,2008M10,564


In [ ]:
dropped_duplicate = df.drop_duplicates()

In [ ]:

dropped_duplicate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3914 entries, 0 to 3919
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Comunidades y Ciudades Autnomas  3914 non-null   object
 1   Provincias                       3914 non-null   object
 2   Periodo                          3914 non-null   object
 3   Total                            3914 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 152.9+ KB


# **Standardization of Data**

In [ ]:
#M seguida de X palabras se reemplaza por ""
df["Periodo"] = df["Periodo"].str.replace(r"M.*$", "")

df['Periodo'] = df['Periodo'].astype(int)

#Cualquier dígito y luego un espacio se reemplaza por ""
df["Provincias"] = df["Provincias"].str.replace(r"\d+\s", "")


<ipython-input-412-1ed12b01f1c4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Periodo"] = df["Periodo"].str.replace(r"M.*$", "")
<ipython-input-412-1ed12b01f1c4>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Provincias"] = df["Provincias"].str.replace(r"\d+\s", "")


In [ ]:
df.to_csv("/content/drive/MyDrive/csv/6150bsc_CLEANED.csv", index=False)